# <span style="color:blue">Amostragem de Sinais</span>

Computadores com placas de áudio podem ser usados para explorar aspectos de <B>AMOSTRAGEM</B> e <B>ALIASING</B>. <P>Neste laboratório vamos gerar sinais e verificar os efeitos da reprodução com diferentes frequências de amostragem.

### Sinal Básico

Verifique se o <i>script</i> seguinte funciona como esperado, ou seja, se um tom LÁ (440 Hz) é reproduzido durante 2 segundos.

In [9]:
from numpy import arange, cos, pi, int8, fft
from pyaudio import PyAudio
from pylab import plot, show
from scipy.io import loadmat

def toca(tom,fs=8000):
  x = PyAudio()
  fluxo = x.open(format=x.get_format_from_width(1),
                 channels=1, rate=fs, output=True)
  amostras = ((tom + 1) * 127.5).astype(int8)
  fluxo.write(amostras.tostring())


Fs = 8000                     # frequência de amostragem
Ts = 1./Fs                    # intervalo de amostragem
tfinal = 1                    # qtde de tempo
n = arange(0,tfinal/Ts)       # índice temporal
ftom = 440                    # freq. do LA central (’A’)
tom = cos(2*pi*ftom*n*Ts)     # gera amostras do sinal senoidal
toca(tom,Fs)                  # toca o sinal (em 8000 sa/seg)

# Fonte: github.com/lneuhaus/pysine/blob/master/pysine/pysine.py

A partir dos comentários no <i>script</i>, deve estar claro o que está acontecendo. Nós configuramos os parâmetros do sistema, geramos amostras do sinal e o reproduzimos no alto-falante. Por padrão, o comando de som é reproduzido em 8000 amostras/s. <p>Experimente amplitudes diferentes para o cosseno; escolha uma amplitude que proporcione um volume de som  confortável pois você vai ouvir esse sinal muitas vezes. <p>Plote o espectro de frequências do tom gerado (valor absoluto da FFT do sinal: <span style="font-family:Courier New; font-size:1em;">*plot(abs(fft(tom)))*</span> ). <p>Explique as abscissas com **valores** e **porque** esses valores aparecem nessas abscissas.

### Subamostragem

O sinal pode ser subamostrado pegando uma amostra e desprezando a seguinte...

In [11]:
tom2 = tom[::2]
toca(tom2,Fs)                 # reproduz o sinal na freq. original

Para efeito de comparação toque esse sinal com a metade da frequência original:

In [14]:
toca(tom2,Fs/2)               # reproduz o sinal numa taxa reduzida

Como soa o sinal <span style="font-family:Courier New; font-size:1em;">tom2</span>? Como se compara a frequência deste sinal com a do primeiro sinal? O que se percebe ao ouvir ambos tons? Trace o espectro do sinal <span style="font-family:Courier New; font-size:1em;">tom2</span> nos moldes do que foi feito para o sinal <span style="font-family:Courier New; font-size:1em;">tom</span>. Explique que caixas têm picos e por quê. <p>Explique as abscissas com **valores** e **porque** esses valores aparecem nessas abscissas

### Outras Decimações

Vamos experimentar outras decimações, ouvindo e plotando os espectros dos sinais decimandos. <p>Em particular, subamostre o sinal por 3,5,8,9,10,15. O que acontece na decimação a partir do fator 9? Por que?

In [5]:
fatores = [2,3,5,7,8,9,10]
for f in fatores:
    print 'Decimando por %d. ' % f,
    raw_input('Pressione [Enter] p/ iniciar\n')   # aguardando a tecla [Enter]
    tomdec = tom[::f]                             # tom decimado
    plot(abs(fft.fft(tomdec))); show()
    toca(tomdec,Fs)                               # reproduz o tom decimado


Decimando por 3. Pressione [Enter] p/ iniciar

 Decimando por 5. Pressione [Enter] p/ iniciar

 Decimando por 8. Pressione [Enter] p/ iniciar

 Decimando por 9. Pressione [Enter] p/ iniciar

 Decimando por 10. Pressione [Enter] p/ iniciar

 Decimando por 15. Pressione [Enter] p/ iniciar




Descreva exatamente que frequência é produzida por cada um dos fatores de decimação. Descreva o que está acontecendo quando o sinal começa a diminuir em frequência. Como chamamos isso? Responda levando em consideração o comportamento dos espectros.  Os picos mudam como esperado? Você observa algum vazamento espectral nesses gráficos?

### Mudança da taxa de reprodução

Agora vamos alterar a taxa (freq.) de reprodução.

In [6]:
toca(tom,Fs/2)

O comando anterior reproduz o sinal em 4000 amostras/seg (ao invés do padrão de 8000 amostras/seg). Com o que se parece o som reproduzido? Porque? <p>Experimente reproduzir o tom em taxas do tipo: Fs, 1.1\*Fs, 0.9\*Fs, 2\*Fs, Fs/2, Fs/3, Fs/4.
(escreva um pequeno script para facilitar a conclusão dessa tarefa.) <p>Descreva como o som produzido muda com estas taxas de amostragem, e porque? 

### Sinal Gorjeio (<i>Chirp</i>)

Agora vamos experimentar um sinal de gorjeio de pássaro, o qual muda sua frequência instantânea com o tempo. <p>Queremos um sinal que mude sua frequência no tempo, de modo que no tempo $t = 0$ a frequência seja $f_1$ Hz e no tempo $t = t_f$ a frequência seja $f_2$ Hz, com variação linear na frequência em função do tempo. Tal sinal é chamado de '*chirp*' linear. Para configurar os dados para tal sinal, precisamos dar uma olhada na relação entre frequência e fase. <p>Considere um sinal

$s(t)=cos(2\pi f_{0}t)$

O argumento para a função cosseno é sempre a fase (sem dimensão). Neste caso, o sinal deste cosseno é $\theta(t)=2πf_0t$. Observe que a frequência do sinal pode ser calculada por:

$$\frac{1}{2\pi} \frac{d\theta(t)}{dt}=f_0$$

Nesse caso, a frequência é constante. 
<p>Mais genericamente, podemos ter uma função de fase que não varia linearmente com o tempo, o que leva a uma frequência variante no tempo. Em geral, para uma função de fase $\theta(t)$ definimos como *frequência instantânea*:

$$f(t)=\frac{1}{2\pi} \frac{d\theta(t)}{dt}\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;(1)$$

Agora vamos definir como queremos nossa frequência instantânea. Façamos $f(t)$ denotar a frequência como uma função do tempo. Queremos $f(0)=f_1$ e $f(t_f)=f_2$, com variação linear entre elas. Podemos então escrever:

$$f(t)=f_1+\frac{f_2-f_1}{t_f}t\;\;$$  ou   $$\;\;f(t)=f_1+m.t\;\;$$ onde $$\;\;m=\frac{f_2-f_1}{t_f}t$$

Agora vamos usar isso no contexto da *frequência instantânea* definida na equação (1):

$$\frac{1}{2\pi} \frac{d\theta(t)}{dt}=f_1+m.t$$

ou $$\theta(t)=2\pi(f_1t+\frac{1}{2}m.t^2)$$

 Este é o argumento para a função cosseno que geraria esse sinal. Ou seja, nós usaríamos:

$$s(t) = cos(\theta(t)) = cos(2\pi t(f_1 + \frac{1}{2}m.t))$$

Observe que a quantidade que multiplica o tempo $t$ é $$f_1+\frac{m.t}{2}$$

No código abaixo, chamamos isso de frequência, embora não seja estritamente a frequência instantânea

In [ ]:
Fs = 8000
Ts = 1./Fs
tfinal = 4                   # tempo final
t = arange(0,tfinal,Ts)      # base temporal
f1 = 440                     # freq. inicial do gorjeio
f2 = 1000                    # freq. final do gorjeio
m = (f2-f1)/tfinal           # inclinação do gorjeio
f = f1 + m*t/2               # frequência variante com o tempo (linear)
gorjeio = cos(2*pi*f*t)

In [3]:
toca(gorjeio,Fs);

Explique o que está acontecendo e por que isso funciona. 
<p>Agora mude as frequências inicial e final para $f_1$ = 2000 Hz e $f_2$ = 8000 Hz. Gere um gráfico da frequência e toque o sinal como antes. Qual é a frequência final percebida? Por que a frequência aumenta e depois diminui?

In [ ]:
# Seu código
f1 = 2000                    # freq. inicial do gorjeio
f2 = 8000                    # freq. final do gorjeio
m = (f2-f1)/tfinal           # inclinação do gorjeio
f = f1 + m*t/2               # frequência variante com o tempo (linear)
gorjeio = cos(2*pi*f*t)
toca(gorjeio,Fs);

In [8]:
plot(abs(fft.fft(gorjeio))); show()

### Aliasing na Música

Agora vamos tentar o efeito aliasing numa música real. Existe um arquivo no sistema conhecido como handel, que tem um pedaço do Coro Aleluia. Você pode carregá-lo (na variável chamada y) e reproduzi-lo

In [15]:
handel = loadmat("handel.mat")
print handel['y']

[[ 0.        ]
 [-0.00615681]
 [-0.07503608]
 ..., 
 [ 0.22741703]
 [ 0.15045695]
 [ 0.        ]]


In [18]:
aleluia = handel['y']
Fs = 8192
toca(aleluia,Fs)

Para obter o efeito que o aliasing pode ter, experimente os seguintes comandos: 

In [21]:
toca(aleluia[::2], Fs) 

In [22]:
toca(aleluia[::2], Fs/2); 

In [23]:
toca(aleluia[::3], Fs/3); 

In [ ]:
toca(aleluia[::4], Fs/4); 

In [24]:
toca(aleluia[::5], Fs/5);

Descreva o efeito que esses comandos têm na reprodução da música e por que eles ocorrem. (Por exemplo, explique por que você obtém o coro de macacos no primeiro.) Por que tanto a decimação (como aleluia[::4]) e a alteração da taxa de amostragem (como Fs/4) são necessárias para manter as coisas corretamente.

By **Prof. Cláudio A. Fleury**, Fev/19.